In [1]:
import numpy as np
from pandas import DataFrame as df
import pandas as pd

### Load validation set in tensorflow format

In [2]:
def load_dataset(path):
    data = pd.read_pickle(path)
    n_features = data['clip_features'][0].shape[-1]
    class_names = data['y'].unique()
    y = np.array([np.where(class_names == e)[0][0] for e in data['y']])
    x = np.concatenate(data['clip_features'].to_numpy())
    print(f'Loaded {len(x)} instances from {path}.')
    print(f'X shape {x.shape} and y shape {y.shape} with labels:\n{data["y"].value_counts()}.')
    train_ds = tf.data.Dataset.from_tensor_slices((x, y)).batch(16)
    return train_ds, class_names, n_features
validation_ds, class_names, n_features = load_dataset('../robot_or_brain_combined_data/validation.pk')


Loaded 750 instances from ../robot_or_brain_combined_data/validation.pk.
X shape (750, 512) and y shape (750,) with labels:
None of the above                  267
Acting or Performing machine       168
Collaborative or Interactive AI    109
Thinking machine                    94
Complex AI                          87
Mysterious AI                       13
Superior human                       8
Learning or recognition machine      4
Name: y, dtype: int64.


NameError: name 'tf' is not defined

In [ ]:
class_list = class_names

In [ ]:
trues = [class_list[int(y)] for _x, y in validation_ds.unbatch()]

### Print class counts for each split

In [ ]:
df([y for y in trues]).value_counts()

Some of these counts are horrible. The number of examples in 'Learning or recognition machine', 'Superior human' and 'Mysterious AI' are really too small. Also, the 'None of the above' class is very large, although that's not really a problem.

### Load our trained model

In [ ]:
import tensorflow as tf
# model = tf.keras.models.load_model('./fine_tuned_model_3m6herki')
# model = tf.keras.models.load_model('./fine_tuned_model_3f0vzk68')
# model = tf.keras.models.load_model('./fine_tuned_model_qdesgan9')
model = tf.keras.models.load_model('./clip_features_model_kuw3ehqp')

### Make predictions

In [ ]:
logits = model.predict(validation_ds)
predicted = [class_list[v] for v in np.argmax(logits, 1)]
from pandas import DataFrame as df
print(df(predicted).value_counts())
print(df(trues).value_counts())

### Performance metrics for ResNet50 based classifier

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
confusion = confusion_matrix(trues, predicted)
disp = ConfusionMatrixDisplay(confusion_matrix=confusion,                               display_labels=class_list)
_ = disp.plot(cmap='Greys', xticks_rotation='vertical')


In [ ]:
from utils import print_performance_metrics

print_performance_metrics(trues, predicted, list(class_list))
